In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import logging

# Prétraitement et modélisation
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Sauvegarde des modèles et des métriques
import joblib
import json

# Ignorer les warnings
import warnings
warnings.filterwarnings('ignore')


In [40]:
# Création des dossiers pour organiser les artefacts
folders = ['data', 'figures', 'models', 'metrics', 'logs']
for folder in folders:
    os.makedirs(folder, exist_ok=True)


In [41]:
# Configuration du logger
logging.basicConfig(
    filename='logs/project_logs.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

logging.info('Initialisation du projet de prédiction du churn client.')


In [42]:
# Chargement des données depuis un fichier CSV
data = pd.read_csv('data.csv')

# Affichage des premières lignes et enregistrement
data.head().to_csv('data/data_preview.csv', index=False)
logging.info('Aperçu des données enregistré dans data/data_preview.csv')

# Informations sur le DataFrame
with open('data/data_info.txt', 'w') as f:
    data.info(buf=f)
logging.info('Informations sur les données enregistrées dans data/data_info.txt')

# Statistiques descriptives et enregistrement
data.describe().to_csv('data/data_describe.csv')
logging.info('Statistiques descriptives enregistrées dans data/data_describe.csv')


In [43]:
# Nettoyage de 'TotalCharges' si nécessaire
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)

# Encodage de la variable cible
data['Churn_encoded'] = data['Churn'].map({'Yes': 1, 'No': 0})


In [44]:
# Liste des variables catégoriques
categorical_vars = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
                    'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                    'Contract', 'PaperlessBilling', 'PaymentMethod']

# Paramètres pour les sous-graphiques
n_cols = 3
n_rows = int(np.ceil(len(categorical_vars) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for idx, var in enumerate(categorical_vars):
    sns.countplot(x=var, hue='Churn', data=data, ax=axes[idx])
    axes[idx].set_title(f'Distribution de {var} par Churn')
    axes[idx].set_xlabel(var)
    axes[idx].set_ylabel('Nombre')
    axes[idx].legend(title='Churn', loc='upper right')

# Supprimer les axes vides
for ax in axes[len(categorical_vars):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.savefig('figures/categorical_barplots.png')
plt.close()
logging.info('Barplots des variables catégoriques enregistrés dans figures/categorical_barplots.png')


In [45]:
# Variables numériques
numerical_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']

n_cols = 3
n_rows = int(np.ceil(len(numerical_vars) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))
axes = axes.flatten()

for idx, var in enumerate(numerical_vars):
    sns.boxplot(x='Churn', y=var, data=data, ax=axes[idx])
    axes[idx].set_title(f'Boxplot de {var} par Churn')
    axes[idx].set_xlabel('Churn')
    axes[idx].set_ylabel(var)

# Supprimer les axes vides
for ax in axes[len(numerical_vars):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.savefig('figures/numerical_boxplots.png')
plt.close()
logging.info('Boxplots des variables numériques enregistrés dans figures/numerical_boxplots.png')


In [46]:
# Matrice de corrélation
corr_matrix = data[numerical_vars + ['Churn_encoded']].corr()

# Heatmap de la matrice de corrélation
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Matrice de Corrélation')
plt.savefig('figures/correlation_matrix.png')
plt.close()
logging.info('Matrice de corrélation enregistrée dans figures/correlation_matrix.png')


In [47]:
# Séparation des features et de la cible
X = data.drop(['Churn', 'customerID', 'Churn_encoded'], axis=1)
y = data['Churn_encoded']

# Identification des variables binaires et catégoriques
binary_vars = [col for col in X.columns if X[col].nunique() == 2]
multi_category_vars = [col for col in X.columns if X[col].dtype == 'object' and col not in binary_vars]


In [48]:
# Création du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_vars),
        ('bin', OrdinalEncoder(), binary_vars),
        ('cat', OneHotEncoder(drop='first'), multi_category_vars)
    ],
    remainder='drop'
)


In [62]:
# Application du préprocesseur sur l'ensemble des données
X_normalized = preprocessor.fit_transform(X)

# Récupération des noms de colonnes après encodage
num_features = numerical_vars
bin_features = binary_vars
cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out(multi_category_vars)
all_features = np.concatenate([num_features, bin_features, cat_features])

# Conversion en DataFrame
X_normalized_df = pd.DataFrame(X_normalized, columns=all_features)

# Enregistrement de la DataFrame normalisée
X_normalized_df.to_csv('data/X_normalized.csv', index=False)
logging.info('DataFrame normalisée enregistrée dans data/X_normalized.csv')


In [63]:
# Division initiale en entraînement et temporaire (70% entraînement, 30% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Division du temporaire en validation et test (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

logging.info(f"Taille de l'entraînement: {X_train.shape}")
logging.info(f"Taille de la validation: {X_val.shape}")
logging.info(f"Taille du test: {X_test.shape}")


In [64]:
# Pipelines pour les modèles sans hyperparamètres
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

pipeline_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(probability=True, random_state=42))
])

logging.info('Pipelines scikit-learn créés pour chaque modèle.')


In [65]:
# Paramètres pour la Régression Logistique
param_grid_lr = {
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__penalty': ['l2'],
    'classifier__solver': ['lbfgs']
}

# Paramètres pour la Forêt Aléatoire
param_grid_rf = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

# Paramètres pour le SVM
param_grid_svc = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['rbf', 'linear'],
    'classifier__gamma': ['scale', 'auto']
}


In [66]:
def fine_tune_model(pipeline, param_grid, X_train, y_train, model_name):
    logging.info(f"Début du fine-tuning pour {model_name}")
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    logging.info(f"Meilleurs paramètres pour {model_name}: {grid_search.best_params_}")
    return grid_search.best_estimator_


In [67]:
# Fine-tuning pour la Régression Logistique
best_model_lr = fine_tune_model(pipeline_lr, param_grid_lr, X_train, y_train, 'LogisticRegression')

# Fine-tuning pour la Forêt Aléatoire
best_model_rf = fine_tune_model(pipeline_rf, param_grid_rf, X_train, y_train, 'RandomForest')

# Fine-tuning pour le SVM
best_model_svc = fine_tune_model(pipeline_svc, param_grid_svc, X_train, y_train, 'SVC')


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [68]:
def plot_confusion_matrix(cm, class_names, title='Confusion matrix', cmap=plt.cm.Blues, file_name='confusion_matrix.png'):
    """
    Trace la matrice de confusion personnalisée.

    Args:
        cm (np.ndarray): Matrice de confusion.
        class_names (list): Liste des noms des classes.
        title (str): Titre du graphique.
        cmap (matplotlib.colors.Colormap): Colormap pour le graphique.
        file_name (str): Nom du fichier pour enregistrer le graphique.
    """
    logging.info(f"Plotting confusion matrix for {title}")
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        percentage = 100 * cm[i, j] / cm[i, :].sum() if cm[i, :].sum() != 0 else 0
        plt.text(j, i, f'{cm[i, j]} ({percentage:.2f}%)',
                 horizontalalignment="center", verticalalignment='center',
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(file_name)
    plt.close()


In [75]:
def evaluate_model(model, X_val, y_val, model_name):
    logging.info(f'Évaluation du modèle {model_name}')
    
    # Prédictions
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1]
    
    # Calcul des métriques
    report = classification_report(y_val, y_pred, output_dict=True)
    auc = roc_auc_score(y_val, y_proba)
    fpr, tpr, thresholds = roc_curve(y_val, y_proba)
    
    # Enregistrement du rapport de classification
    with open(f'metrics/classification_report_{model_name}.txt', 'w') as f:
        f.write(classification_report(y_val, y_pred))
    logging.info(f'Rapport de classification enregistré pour {model_name}')
    
    # Matrice de confusion
    cm = confusion_matrix(y_val, y_pred)
    class_names = ['No', 'Yes']
    plot_confusion_matrix(cm, class_names, title=f'Matrice de Confusion - {model_name}', file_name=f'figures/confusion_matrix_{model_name}.png')
    
    # Courbe ROC
    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f'Courbe ROC - {model_name}')
    plt.xlabel('Taux de Faux Positifs')
    plt.ylabel('Taux de Vrais Positifs')
    plt.legend(loc='lower right')
    plt.savefig(f'figures/roc_curve_{model_name}.png')
    plt.close()
    logging.info(f'Courbe ROC enregistrée pour {model_name}')
    
    # Sauvegarder les métriques
    metrics = {
        'classification_report': report,
        'auc_roc': auc
    }
    with open(f'metrics/metrics_{model_name}.json', 'w') as f:
        json.dump(metrics, f, indent=4)
    logging.info(f'Métriques enregistrées pour {model_name}')
    
    return report, auc


In [76]:
# Dictionnaires pour stocker les rapports et les AUC
models_reports = {}
models_auc = {}

# Évaluation du modèle Logistic Regression
report_lr, auc_lr = evaluate_model(best_model_lr, X_val, y_val, 'LogisticRegression')
models_reports['LogisticRegression'] = report_lr
models_auc['LogisticRegression'] = auc_lr

# Évaluation du modèle Random Forest
report_rf, auc_rf = evaluate_model(best_model_rf, X_val, y_val, 'RandomForest')
models_reports['RandomForest'] = report_rf
models_auc['RandomForest'] = auc_rf

# Évaluation du modèle SVC
report_svc, auc_svc = evaluate_model(best_model_svc, X_val, y_val, 'SVC')
models_reports['SVC'] = report_svc
models_auc['SVC'] = auc_svc


In [77]:
# Sauvegarde des modèles optimisés
joblib.dump(best_model_lr, 'models/model_LogisticRegression.pkl')
joblib.dump(best_model_rf, 'models/model_RandomForest.pkl')
joblib.dump(best_model_svc, 'models/model_SVC.pkl')

logging.info('Modèles optimisés sauvegardés dans le dossier models.')


In [78]:
# Création d'une liste pour stocker les rapports
classification_reports = []

for model_name, report in models_reports.items():
    report_df = pd.DataFrame(report).transpose()
    report_df['model'] = model_name
    report_df['metric'] = report_df.index
    classification_reports.append(report_df)

# Concaténer tous les rapports
all_reports_df = pd.concat(classification_reports, axis=0)

# Réorganisation des colonnes
cols = ['model', 'metric'] + [col for col in all_reports_df.columns if col not in ['model', 'metric']]
all_reports_df = all_reports_df[cols]

# Enregistrement dans un fichier CSV
all_reports_df.to_csv('metrics/all_classification_reports.csv', index=False)
logging.info('Tous les rapports de classification enregistrés dans metrics/all_classification_reports.csv')


In [79]:
# Poids des métriques
weights = {
    'precision': 0.25,
    'recall': 0.25,
    'f1-score': 0.25,
    'auc_roc': 0.25
}

benchmark_results = {}

for model_name, report in models_reports.items():
    # Récupérer les scores pour la classe positive ('1')
    metrics_class_1 = report['1']
    auc = models_auc[model_name]
    
    # Calcul du score final pondéré
    final_score = (
        weights['precision'] * metrics_class_1['precision'] +
        weights['recall'] * metrics_class_1['recall'] +
        weights['f1-score'] * metrics_class_1['f1-score'] +
        weights['auc_roc'] * auc
    )
    
    benchmark_results[model_name] = {
        'precision': metrics_class_1['precision'],
        'recall': metrics_class_1['recall'],
        'f1-score': metrics_class_1['f1-score'],
        'auc_roc': auc,
        'final_score': final_score
    }

# Création de la DataFrame des résultats
benchmark_df = pd.DataFrame(benchmark_results).T

# Enregistrement de la DataFrame des scores
benchmark_df.to_csv('metrics/benchmark_results.csv')
logging.info('Résultats du benchmarking enregistrés dans metrics/benchmark_results.csv')

# Affichage du meilleur modèle
best_model_name = benchmark_df['final_score'].idxmax()
print(f"Le meilleur modèle est : {best_model_name}")
logging.info(f'Le meilleur modèle est : {best_model_name}')


Le meilleur modèle est : LogisticRegression


In [80]:
# Comparaison des modèles sur les métriques
benchmark_df[['precision', 'recall', 'f1-score', 'auc_roc']].plot(kind='bar', figsize=(10, 6))
plt.title('Comparaison des Modèles sur les Différentes Métriques')
plt.ylabel('Score')
plt.xlabel('Modèle')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('figures/model_comparison_metrics.png')
plt.close()
logging.info('Figure de comparaison des métriques enregistrée dans figures/model_comparison_metrics.png')

# Figure pour le score final
benchmark_df['final_score'].plot(kind='bar', figsize=(8, 6))
plt.title('Score Final Pondéré des Modèles')
plt.ylabel('Score Final')
plt.xlabel('Modèle')
plt.tight_layout()
plt.savefig('figures/model_final_scores.png')
plt.close()
logging.info('Figure des scores finaux enregistrée dans figures/model_final_scores.png')
